In [1]:
import pandas as pd

In [ ]:
expedia_df = pd.read_csv('Data/combined_expedia_results.csv')
booking_df = pd.read_csv('TaskB - Booking\merged_booking_final.csv')

#clean hotel names
expedia_df['Hotel Name'] = expedia_df['Hotel Name'].str.strip().str.lower()
booking_df['hotel_name'] = booking_df['hotel_name'].str.strip().str.lower()

matched_df = pd.merge(expedia_df[['Hotel Name', 'TTT']]